<a href="https://colab.research.google.com/github/rahiakela/transfer-learning-for-natural-language-processing/blob/main/3-shallow-transfer-learning-for-nlp/2_semi_supervised_learning_with_pretrained_sentence_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Semi-supervised Learning with Pretrained Sentence Embeddings(Higher-Level Representations)

In this notebook, we will cover some prominent shallow transfer learning approaches and concepts. This allows us to explore some major themes in transfer learning, while doing so in the context of relatively simple models of the class of eventual interest, i.e., shallow neural networks.

Roughly speaking, categorization is based on whether transfer occurs between different languages, tasks or data domains. Each of these types of categorization is usually correspondingly referred to as cross-lingual learning, multi-task learning and domain adaptation.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/shallow-transfer-learning.png?raw=1' width='800'/>

The methods we will look at here will involve components that are neural networks in one way or another.these neural networks do not have many
layers. This is the reason why the label “shallow” is appropriate to describe this collection of methods.

A common form of semi-supervised learning that employs pretrained word embeddings such as word2vec that they produce a single vector per word, regardless of context.

We revisit the IMDB movie review sentiment classification. Recall that this example is concerned with classifying movie reviews from IMDB into positive or negative sentiments expressed. It is a prototypical sentiment analysis example that has been used widely in the literature to study many algorithms. We combine feature vectors generated by pretrained word embeddings for each review with some traditional machine learning classification methods, namely random forests and logistic regression.

We then demonstrate that using higher-level embeddings which vectorize bigger sections of text – such as at the sentence-level, paragraphlevel and document-level – can lead to improved performance. The general idea of vectorizing text and then applying a traditional machine learning classification method to the resulting vectors.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/semi-supervised-learning.png?raw=1' width='800'/>

**Multi-task learning**

Subsequently, we introduce the reader to multi-task learning. We demonstrate how one can train a single system simultaneously to perform multiple tasks, email spam classification and IMDB movie review sentiment analysis. 

There are several potential benefits to multi-task learning. By
training a single machine learning model for multiple tasks, a shared representation is learned on a larger and more varied collection of data from the combined data pool, which can lead to performance improvements. Moreover, it has been widely observed that this shared representation has a better ability to generalize to tasks beyond those that were trained on, and
this improvement can be achieved without any increase in model size.

Specifically, we focus on shallow neural multitask learning, where a single additional dense layer, as well as a classification layer, is trained
for each specific task in the setup. Different tasks also share a layer between them, a setup typically referred to as hard-parameter sharing.

**Domain adaptation**

Assume that we are given one source domain, which can be defined as a particular distribution of data for a specific task, and a classifier that has been trained to perform well on data in that domain for that task. The goal of domain adaptation is to modify, or adapt, data in a different target domain in such a way that the pretrained knowledge from the source domain can aid
learning in the target domain. We apply a simple autoencoding approach to “project” samples in the target domain into the source domain feature space.

An autoencoder is a system that learns to reconstruct inputs with very high accuracy, typically by encoding them into an efficient latent representation and learning to decode the said representation efficiently. They have traditionally been heavily used in model reduction applications, since the latent representation is often of smaller dimension than the original space
from which the encoding happens, and the said dimension value can also be picked to strike the right balance of computational efficiency and accuracy.

In the extreme scenario, improvements can be obtained with no labelled data in the target domain being used for training. This is typically referred to as zero-shot domain adaptation, where learning happens with no labeled
data in the target domain.


## Setup

In [ ]:
# install sent2vec
!pip install git+https://github.com/epfml/sent2vec

In [1]:
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import re
import os
import time

from gensim.models import FastText, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier      # random forest classifier library
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import sent2vec

import matplotlib.pyplot as plt
from IPython.display import HTML

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Download IMDB Movie Review Dataset

In [5]:
%%shell

wget -q "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
tar xzf aclImdb_v1.tar.gz

rm -rf aclImdb_v1.tar.gz
rm -rf aclImdb/train/unsup

Let's download sent2vec word Embedding from [Kaggle](https://www.kaggle.com/maxjeblick/sent2vec)

In [7]:
from google.colab import files
files.upload() # upload kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahiakela","key":"484f91b2ebc194b0bff8ab8777c1ebff"}'}

In [8]:
%%shell

mkdir -p ~/.kaggle
mv kaggle.json ~/.kaggle/
ls ~/.kaggle
chmod 600 /root/.kaggle/kaggle.json

# download dataset from kaggle
kaggle datasets download -d maxjeblick/sent2vec/wiki_unigrams.bin
unzip -qq sent2vec.zip
rm -rf sent2vec.zip

kaggle.json
100% 4.43G/4.43G [01:01<00:00, 31.1MB/s]
100% 4.43G/4.43G [01:01<00:00, 77.9MB/s]


## Preprocessing IMDB Movie Data

Before proceeding, we must decide how many samples to draw from each class. We must also decide the maximum number of tokens per email, and the maximum length of each token. This is done by setting the following overarching hyperparameters.

In [2]:
n_sample = 1000   # number of samples to generate in each class
maxtokens = 200    # the maximum number of tokens per document
maxtokenlen = 100  # the maximum length of each token

### Tokenization

Let’s proceed by defining a function to tokenize text by splitting them into 
words.

In [3]:
def tokenize(row):
  if row is None or row is "":
    tokens = ""
  else:
    tokens = row.split(" ")[:maxtokens]
  return tokens

### Remove punctuation and unnecessary characters

**In order to ensure that classification is done based on language content only, we have to remove punctuation marks and other non-word characters from the emails.** We do this by employing regular expressions with the Python regex library. We also normalize words by turning them into lower case.

In [4]:
def reg_expressions(row):
  tokens = []
  try:
    for token in row:
      token = token.lower()          # make all characters lower case
      token = re.sub(r"[\W\d]", "", token)
      token = token[:maxtokenlen]    # truncate all tokens to hyperparameter maxtokenlen
      tokens.append(token)
  except:
    token = ""
    tokens.append(token)
  return tokens

### Stop-word removal

Stop-words are also removed. Stop-words are words that are very common in text but offer no useful information that can be used to classify the text. Words such as is, and, the, are are examples of stop-words. The NLTK library contains a list of 127 English stop-words and can be used to filter our tokenized strings.

In [5]:
stop_words = stopwords.words("english")
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# print(stopwords) # see default stopwords
# it may be beneficial to drop negation words from the removal list, as they can change the positive/negative meaning
# of a sentence
# stopwords.remove("no")
# stopwords.remove("nor")
# stopwords.remove("not")

In [6]:
def stop_word_removal(row):
  token = [token for token in row if token not in stop_words]
  token = filter(None, token)

  return token

## Semi-supervised Learning with Higher-Level Representations

One prominent approach is paragraph vectors or doc2vec, which exploits the concatenation (vs. averaging) of words from pretrained word embeddings in summarizing them. Another is sent2vec, which extends the classic Continuous Bag-of-Words (CBOW) of word2vec – where a shallow network is trained to predict
a word in a sliding window from its context – to sentences by optimizing word and word n-gram embeddings for an accurate averaged representation.

## Load pre-trained sent2vec embedding

Quite naturally, just as in the case of the pretrained word embeddings, the next step is to obtain the pretrained sent2vec sentence embedding to be loaded by the particular implementation/framework installed.

We choose the smallest 600-dimensional embedding `wiki_unigrams.bin`, approximately 5 Gigabytes in size, which captures just the unigram information on Wikipedia.

Now let's load the pre-trained embedding.

In [11]:
# load sent2vec embedding
model = sent2vec.Sent2vecModel()

start = time.time()
model.load_model("wiki_unigrams.bin")
end = time.time()

print("Loading the sent2vec embedding took %d seconds" % (end - start))

Loading the sent2vec embedding took 8 seconds


We found the load time during the first execution to be less than 10 seconds, a notable improvement over the fastText word embedding loading time of over 10 minutes that is worth mentioning before proceeding.

## Extract corresponding vectors from the pretrained word embedding

Next, we define a function to generate vectors for a collection of reviews. It is essentially a simpler form of the function presented in Listing 3.2 for pretrained word embeddings – it is simpler as we do not need to worry about out-of-vocabulary words.

In [12]:
def assemble_embedding_vectors(data):
  out = None
  for item in data:    # Loop through every IMDB review
    vec = model.embed_sentence(" ".join(item))     # Extract embedding vectors for every word in review, now we dont need to handle out-of-vocab words
    if vec is not None:                            # Edge case handling
      if out is not None:
        out = np.concatenate((out, vec), axis=0)    # Concatenate row vector to output Numpy array
      else:
        out = vec
    else:
      pass

  return out

## Preparing and assembling Dataset

In [13]:
# shuffle raw data first
def unison_shuffle_data(data, header):
    p = np.random.permutation(len(header))
    data = data[p]
    header = np.asarray(header)[p]
    return data, header

# load data in appropriate form
def load_data(path):
  data, sentiments = [], []
  for folder, sentiment in (("neg", 0), ("pos", 1)):
    folder = os.path.join(path, folder)
    for name in os.listdir(folder):    # Go through every file in current folder
      with open(os.path.join(folder, name), "r") as reader:
        text = reader.read()
      # Apply tokenization, stopword analysis routines
      text = tokenize(text)
      text = stop_word_removal(text)
      text = reg_expressions(text)
      # Track corresponding text and sentiment labels
      data.append(text)
      sentiments.append(sentiment)
  # Convert to Numpy array
  #print(data)
  data_np = np.array(data)
  #print(data_np[:10])
  data, sentiments = unison_shuffle_data(data_np, sentiments)

  return data, sentiments

In [14]:
train_path = os.path.join("aclImdb", "train")
test_path = os.path.join("aclImdb", "test")
raw_data, raw_header = load_data(train_path)

print(raw_data.shape)
print(len(raw_header))

(25000,)
25000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
# Subsample required number of samples
random_indices = np.random.choice(range(len(raw_header)), size=(n_sample * 2,), replace=False)
data_train = raw_data[random_indices]
header = raw_header[random_indices]

print("DEBUG::data_train::")
print(data_train[:10])

DEBUG::data_train::
[list(['i', 'feel', 'blessed', 'known', 'worst', 'steven', 'seagal', 'movie', 'ever', 'made', 'i', 'knew', 'i', 'something', 'special', 'steven', 'opened', 'mouth', 'someone', 'elses', 'voice', 'came', 'out', 'by', 'middle', 'film', 'eyes', 'beginning', 'hurt', 'i', 'almost', 'falling', 'chair', 'uncontrollable', 'laughterbr', 'br', 'steven', 'steven', 'with', 'ever', 'changing', 'voice', 'totally', 'unbelievable', 'role', 'as', 'always', 'who', 'hell', 'lets', 'people', 'bad', 'nappyhair', 'pony', 'tail', 'mullets', 'forces', 'anyway', 'he', 'also', 'always', 'writes', 'totally', 'unbelievable', 'love', 'interests', 'women', 'least', '', 'years', 'junior', 'the', 'supporting', 'actors', 'look', 'like', 'theyve', 'shot', 'dark', '', 'btw', 'shoot', 'movie', 'dark', 'penlight', 'torch', 'lighting', 'br', 'br', 'this', 'truly', 'abominable', 'every', 'way', 'possible', 'invite', 'friends', 'around', 'make', 'social', 'event', '', 'ones', 'truly', 'special'])
 list(['l

Display sentiments and their frequencies in the dataset, to ensure it is roughly balanced between classes.

In [16]:
unique_elements, counts_elements = np.unique(header, return_counts=True)

print("Sentiments and their frequencies:")
print(unique_elements)
print(counts_elements)

Sentiments and their frequencies:
[0 1]
[1022  978]


We can now use this function to extract sent2vec embedding vectors for each review.

In [17]:
embedding_vectors = assemble_embedding_vectors(data_train) 
print(embedding_vectors)

[[ 0.05124323 -0.1399035   0.182506   ...  0.06421937  0.01265034
   0.27128586]
 [ 0.19040097 -0.04049111  0.07053659 ...  0.07800578 -0.05346423
   0.08981815]
 [ 0.10107018 -0.0192001   0.09538116 ... -0.08886915 -0.04290215
   0.24210128]
 ...
 [-0.04037483 -0.0962265   0.00098962 ...  0.09110203 -0.04778019
   0.10191945]
 [ 0.10662704 -0.09184669 -0.13963842 ...  0.05885526  0.01938395
   0.21178496]
 [ 0.01132368 -0.12341179 -0.03747171 ... -0.09069834 -0.06768979
   0.2237481 ]]


These can now be used as feature vectors for the same logistic regression and random forest.

As the very last step of preparing the sentiment dataset for training by our baseline classifiers, we split it into independent training and testing or validation sets. This will allow us to evaluate the performance of the classifier on a set of data that was not used for training, an important thing
to ensure in machine learning practice. We elect to use 70% of the data for training, and 30% for testing/validation afterwards.

In [18]:
data = embedding_vectors

idx = int(0.7 * data.shape[0])


# 70% of data for training
train_x = data[:idx, :]
train_y = header[:idx]

# remaining 30% for testing
test_x = data[idx:, :]
test_y = header[idx:]

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
print(train_x[:5])
print(len(train_y))
print(train_y[:5])

train_x/train_y list details, to make sure it is of the right form:
1400
[[ 0.05124323 -0.1399035   0.182506   ...  0.06421937  0.01265034
   0.27128586]
 [ 0.19040097 -0.04049111  0.07053659 ...  0.07800578 -0.05346423
   0.08981815]
 [ 0.10107018 -0.0192001   0.09538116 ... -0.08886915 -0.04290215
   0.24210128]
 [-0.0300916   0.07088199 -0.05434231 ...  0.01374236 -0.05029749
   0.1384656 ]
 [ 0.0039798  -0.13911213 -0.07523181 ... -0.04756916 -0.00103771
   0.03802983]]
1400
[0 1 0 0 0]


## Logistic Regression Classifier

Logistic regression models the relationship between a categorical output variable and a set of input variables by estimating probabilities with the logistic function. Assuming the existence of a single input variables x, and a single output binary variable y with associated probability $P(y=1)=p$.

Now, let’s go ahead and build our classifier using the popular library scikit-learn.

In [19]:
def fit(train_x, train_y):
  model = LogisticRegression()

  try:
    model.fit(train_x, train_y)
  except:
    pass
  
  return model

In [20]:
model = fit(train_x, train_y)

predicted_labels = model.predict(test_x)
print("DEBUG::The logistic regression predicted labels are::")
print(predicted_labels)

DEBUG::The logistic regression predicted labels are::
[0 0 1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1
 1 0 1 1 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0
 0 1 0 1 0 1 1 1 1 0 0 0 1 1 1 0 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 0
 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 1 0 0 1
 0 0 1 1 1 0 1 0 1 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 1
 1 0 0 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1
 1 0 1 1 1 1 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 1 1 0
 1 1 1 0 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 1 0 0
 1 1 0 1 0 0 1 0 0 0 1 0 1 1 1 1 0 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0
 1 1 1 0 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 0 1 1 1

In [21]:
acc_score = accuracy_score(test_y, predicted_labels)
print("The logistic regression accuracy score is::")
print(acc_score)

The logistic regression accuracy score is::
0.815


## Random Forests

Random Forests (RFs) provide a practical machine learning method for applying decision trees. It involves generating a very large number of specialized trees and ensembling their outputs. RFs are extremely flexible and widely applicable, making them often the second algorithm practitioners try after logistic regression for baselining.

In [22]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=1, random_state=0)

# Train the Classifier to take the training features and learn how they relate to the training y (spam, not spam?)
start_time = time.time()
clf.fit(train_x, train_y)
end_time = time.time()
print("Training the Random Forest Classifier took %3d seconds"%(end_time-start_time))

Training the Random Forest Classifier took   2 seconds


In [23]:
predicted_labels = clf.predict(test_x)
print("DEBUG::The RF predicted labels are::")
print(predicted_labels)

DEBUG::The RF predicted labels are::
[0 0 1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 1 1
 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0
 1 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 1
 1 1 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1
 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1
 0 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1
 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1
 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 1 1
 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0 1 1 1 0
 1 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 0 0 1 0 1 1 1 1
 1 0 0 1 1 1 0 1 1 1 1 1 0 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 1 1 0 0 1 0 0
 1 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 0 0
 1 1 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 

In [24]:
acc_score = accuracy_score(test_y, predicted_labels)
print("DEBUG::The RF testing accuracy score is::")
print(acc_score)

DEBUG::The RF testing accuracy score is::
0.745


## Conclusions

This yields accuracy scores of 82% and 74% for the logistic regression and random forest classifiers respectively (at the same hyperparameter
values as in the previous section). This value for the logistic regression classifier combined with sent2vec is an improvement on the corresponding values of 63% and 67% respectively for the bag-of-words baseline.